# IMPORT LIBRARIES

In [1]:
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
from env import user, password, host
from scipy import stats
from scipy.stats import levene, ttest_ind
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice
from datetime import datetime
import math
import numpy as np
import os
import pandas as pd
import requests
import seaborn as sns
import statsmodels.api as sm
import time
import warnings
warnings.filterwarnings("ignore")
from pandas_profiling import ProfileReport

In [55]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%

# ACQUIRE

In [2]:
def get_311_data():
    ''' 
    This function reads in a csv held in the same repository folder
    '''
    df = pd.read_csv('data_311.csv')
    return df


In [3]:
df=get_311_data()

# SUMMARIZE DATA 
#### WHAT DO WE HAVE HERE?

In [4]:
# return first df rows
df.head()

,source_id,dept_division,case_id,case_opened_date,case_closed_date,SLA_due_date,case_late,num_days_late,case_closed,service_request_type,SLA_days,case_status,request_address,council_district,dept_name,standardized_dept_name,dept_subject_to_SLA,index,source_username
0,139344,Miscellaneous,1014127378,1/1/18 8:24,1/2/18 7:37,1/3/18 8:24,NO,-1.032870,YES,Dead Animal - Cat,2.0,Closed,"7059 TIMBERCREEK DR, San Antonio, 78227",6,Solid Waste Management,Solid Waste,YES,19,Isis Mendoza
1,139344,Field Operations,1014127380,1/1/18 8:39,1/1/18 12:30,9/26/20 8:39,NO,-998.839687,YES,Stray Animal,999.0,Closed,FREDSBG RD and NB LOOP 410 NW,7,Animal Care Services,Animal Care Services,YES,19,Isis Mendoza
2,139344,Signals,1014127389,1/1/18 9:02,1/1/18 10:12,1/3/18 8:30,NO,-1.928912,YES,Traffic Signal Ops and Maintenance - Emergency,1.977454,Closed,CESAR CHAVEZ W and S ZARZAMORA,5,Trans & Cap Improvements,Trans & Cap Improvements,YES,19,Isis Mendoza
3,139344,Streets,1014127390,1/1/18 9:04,1/3/18 8:54,1/9/18 8:30,NO,-5.983299,YES,Barricade Pick Up,7.976192,Closed,CHUPADERAS N and MORALES,1,Trans & Cap Improvements,Trans & Cap Improvements,YES,19,Isis Mendoza
4,139344,Signals,1014127394,1/1/18 9:15,1/1/18 10:54,1/3/18 8:30,NO,-1.899375,YES,Traffic Signal Ops and Maintenance - Emergency,1.968414,Closed,CALAVERAS N and FREDSBG RD,1,Trans & Cap Improvements,Trans & Cap Improvements,YES,19,Isis Mendoza


#### Looks like a log of calls for city service departments

In [5]:
# return last df rows
df.tail()

,source_id,dept_division,case_id,case_opened_date,case_closed_date,SLA_due_date,case_late,num_days_late,case_closed,service_request_type,SLA_days,case_status,request_address,council_district,dept_name,standardized_dept_name,dept_subject_to_SLA,index,source_username
855264,svcCRMSS,Field Operations,1014127272,12/31/17 18:35,12/31/17 18:45,12/31/17 21:35,NO,-0.118009,YES,Animal Cruelty(Critical),0.125,Closed,FAY AVE and LYELL,5,Animal Care Services,Animal Care Services,YES,133,svcCRMSS
855265,svcCRMSS,Field Operations,1014127283,12/31/17 19:08,12/31/17 22:03,12/31/17 22:08,NO,-0.003299,YES,Animal Cruelty(Critical),0.125,Closed,"207 COVINGTON RD, San Antonio, 78220",2,Animal Care Services,Animal Care Services,YES,133,svcCRMSS
855266,svcCRMSS,Field Operations,1014127299,12/31/17 19:50,12/31/17 19:58,12/31/17 22:50,NO,-0.119606,YES,Animal Cruelty(Critical),0.125,Closed,"928 LEAL ST, San Antonio, 78207",1,Animal Care Services,Animal Care Services,YES,133,svcCRMSS
855267,svcCRMSS,Field Operations,1014127312,12/31/17 21:07,12/31/17 21:12,1/1/18 0:07,NO,-0.121493,YES,Animal Cruelty(Critical),0.125,Closed,PALMETTO WAY and LEXI PETAL,0,Animal Care Services,Animal Care Services,YES,133,svcCRMSS
855268,svcCRMSS,Field Operations,1014127322,12/31/17 22:04,12/31/17 22:09,1/1/18 1:04,NO,-0.121713,YES,Animal Cruelty(Critical),0.125,Closed,"233 THEO AVE E, San Antonio, 78214",3,Animal Care Services,Animal Care Services,YES,133,svcCRMSS


In [6]:
#list column names
df.columns

Index(['source_id', 'dept_division', 'case_id', 'case_opened_date',
       'case_closed_date', 'SLA_due_date', 'case_late', 'num_days_late',
       'case_closed', 'service_request_type', 'SLA_days', 'case_status',
       'request_address', 'council_district', 'dept_name',
       'standardized_dept_name', 'dept_subject_to_SLA', 'index',
       'source_username'],
      dtype='object')

#### Might need to see if columns have REDUNDANCY and see if column will be useful 

In [7]:
# give numerical date, stats, min, max, quartile info
df.describe()

,case_id,num_days_late
count,8.552690e+05,855235.000000
mean,1.013968e+09,-49.177286
std,4.422757e+05,177.036916
min,1.013226e+09,-1417.000602
25%,1.013581e+09,-37.992980
50%,1.013948e+09,-5.013958
75%,1.014350e+09,-1.827373
max,1.014760e+09,519.698079


In [54]:
df.describe(include='all')

,source_id,dept_division,case_id,case_opened_date,case_closed_date,SLA_due_date,case_late,num_days_late,case_closed,service_request_type,SLA_days,case_status,request_address,council_district,dept_name,standardized_dept_name,dept_subject_to_SLA,index,source_username
count,855269,855269,8.552690e+05,855269,836936,855235,855269,855235.000000,855269,855269,855235.0,855269,855269,855269.0,855071,855269,855268,855268.0,855261
unique,164,39,NaN,342441,297942,302669,2,NaN,2,341,149039.0,7,260062,24.0,11,10,3,214.0,140
top,CRM_Listener,Code Enforcement,NaN,6/18/18 2:21,9/6/17 12:00,1/5/18 8:30,NO,NaN,YES,No Pickup,5.0,Closed,"10133 FIGARO CANYON, San Antonio, 78251",1.0,Code Enforcement Services,DSD/Code Enforcement,YES,58.0,CRM_Listener
freq,195416,274298,NaN,35,525,1460,759122,NaN,836936,89210,72467.0,836928,5150,96470.0,321976,323571,855227,121562.0,195408
mean,NaN,NaN,1.013968e+09,NaN,NaN,NaN,NaN,-49.177286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,4.422757e+05,NaN,NaN,NaN,NaN,177.036916,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,1.013226e+09,NaN,NaN,NaN,NaN,-1417.000602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,1.013581e+09,NaN,NaN,NaN,NaN,-37.992980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,1.013948e+09,NaN,NaN,NaN,NaN,-5.013958,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,1.014350e+09,NaN,NaN,NaN,NaN,-1.827373,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# list number columns and rows, data types,and non null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 855269 entries, 0 to 855268
Data columns (total 19 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   source_id               855269 non-null  object 
 1   dept_division           855269 non-null  object 
 2   case_id                 855269 non-null  int64  
 3   case_opened_date        855269 non-null  object 
 4   case_closed_date        836936 non-null  object 
 5   SLA_due_date            855235 non-null  object 
 6   case_late               855269 non-null  object 
 7   num_days_late           855235 non-null  float64
 8   case_closed             855269 non-null  object 
 9   service_request_type    855269 non-null  object 
 10  SLA_days                855235 non-null  object 
 11  case_status             855269 non-null  object 
 12  request_address         855269 non-null  object 
 13  council_district        855269 non-null  object 
 14  dept_name           

#### Most are object but case_id is an integer and num_days_late is a float. Also, I might want to organize it by the case_id, or by date, maybe by case_opened_date.

In [9]:
# number null values in each column
df.isnull().sum()

source_id                     0
dept_division                 0
case_id                       0
case_opened_date              0
case_closed_date          18333
SLA_due_date                 34
case_late                     0
num_days_late                34
case_closed                   0
service_request_type          0
SLA_days                     34
case_status                   0
request_address               0
council_district              0
dept_name                   198
standardized_dept_name        0
dept_subject_to_SLA           1
index                         1
source_username               8
dtype: int64

#### But it isn't a large amount, so I will consider dropping columns with nulls

#### NOTE for the cells below. I know that the following is a bit much, but I really need to see how the data is formatted

In [10]:
# unique values of column
df.source_id.unique() 

array([139344, 124405, 135723, 139342, 136202, 141549, 140987, 132408,
       119555, 141239, 140508, 140637, 141256, 140436, 139868, 138650,
       120752, 138793, 139345, 138810, 138605, 100137, 139807, 141954,
       141752, 142659, 142673, 142738, 106463, 142989, 119868, 142991,
       119403, 137943, 136979, 140509, 140507, 140992, 140991, 141241,
       141240, 103582, 142088, 142087, '138810', '141241', '135723',
       '139344', '142087', '138793', '119555', '141549', '141239',
       '132408', '140987', '140436', '140508', '142088', '139345',
       '136202', '140637', '120752', '139868', '124405', '138605',
       '100137', '141256', '139342', 'svcCRMSS', 'ns16326', 'svcCRMLS',
       'CRM_Listener', 'rs16746', 'mt13131', 'NO10960', 'jw10936',
       'mc21309', 'cc17850', 'mp21218', 'MW16328', 'dc12381', 'gp20685',
       'eg16317', 'dl05036', 'mt23152', 'ss21394', 'ss09159', 'sg22264',
       'bn26322', 'sv24848', 'dd26293', 'FC24472', 'FR26584', 'gg14829',
       'DS15677',

In [11]:
# number of unique values
df.source_id.nunique() 

164

In [12]:
# unique values of column
df.dept_division.unique() 

array(['Miscellaneous', 'Field Operations', 'Signals', 'Streets',
       'Code Enforcement', 'Traffic Engineering Design', 'Brush',
       'Signs and Markings', 'Waste Collection', '311 Call Center',
       'Graffiti', 'Storm Water', 'Engineering Division',
       'Dangerous Premise', 'Trades', 'Food Establishments',
       'Facility License', 'Solid Waste', "Director's Office Horizontal",
       'Clean and Green', 'Vector', 'Shops', 'District 2', 'Tree Crew',
       'District 1', 'District 6', 'District 8', 'District 9',
       'District 7', 'Clean and Green Natural Areas', 'District 10',
       'Graffiti (IntExp)', 'District 3', 'Reservations',
       'Dangerous Premise (IntExp)', 'Code Enforcement (IntExp)',
       'Code Enforcement (Internal)', 'Shops (Internal)',
       'Storm Water Engineering'], dtype=object)

In [13]:
# number of unique values
df.dept_division.nunique()

39

In [14]:
# unique values of column
df.case_id.unique() 

array([1014127378, 1014127380, 1014127389, ..., 1014127299, 1014127312,
       1014127322])

In [15]:
# number of unique values
df.case_id.nunique()

841592

In [16]:
# unique values of column
df.case_opened_date.unique() 

array(['1/1/18 8:24', '1/1/18 8:39', '1/1/18 9:02', ..., '12/31/17 19:08',
       '12/31/17 19:50', '12/31/17 21:07'], dtype=object)

In [17]:
# number of unique values
df.case_opened_date.nunique()

342441

In [18]:
# unique values of column
df.case_closed_date.unique() 

array(['1/2/18 7:37', '1/1/18 12:30', '1/1/18 10:12', ...,
       '12/31/17 22:03', '12/31/17 19:58', '12/31/17 22:09'], dtype=object)

In [19]:
# number of unique values
df.case_closed_date.nunique()

297942

In [20]:
# unique values of column
df.SLA_due_date.unique() 

array(['1/3/18 8:24', '9/26/20 8:39', '1/3/18 8:30', ...,
       '12/31/17 22:50', '1/1/18 0:07', '1/1/18 1:04'], dtype=object)

In [21]:
# number of unique values
df.SLA_due_date.nunique()

302669

In [22]:
# unique values of column
df.case_late.unique() 

array(['NO', 'YES'], dtype=object)

In [23]:
# number of unique values
df.case_late.nunique()

2

In [24]:
# unique values of column
df.num_days_late.unique() 

array([-1.03287037e+00, -9.98839687e+02, -1.92891204e+00, ...,
       -5.96753472e+00, -6.88657407e-01, -1.21712963e-01])

In [25]:
# number of unique values
df.num_days_late.nunique()

599211

In [26]:
# unique values of column
df.case_closed.unique() 

array(['YES', 'NO'], dtype=object)

In [27]:
# number of unique values
df.case_closed.nunique()

2

In [28]:
# unique values of column
df.service_request_type.unique() 

array(['Dead Animal - Cat', 'Stray Animal',
       'Traffic Signal Ops and Maintenance - Emergency',
       'Barricade Pick Up', 'Animal Neglect', 'Dead Animal - Dog',
       'Aggressive Animal(Non-Critical)', 'Trapped/Confined Animal',
       'Dead Animal - Misc', 'Public Nuisance(Own Animal)',
       'Officer Standby', 'Injured Animal(Critical)',
       'Signal Timing Modification', 'Minimum Housing-Owner Occupied',
       'Minimum Housing-Tenant Occupied', 'Animal Permits Request',
       'Flashing Beacon New Request',
       'Illegal Dumping (Non- residential areas)', 'Traffic Sign Faded',
       'Overgrown Yard/Trash', 'Solid Waste Fees and Charges',
       'Base/Pavement Repair', 'Cart Delivery',
       'Vacant Lot/Overgrown/Trash', 'Water No Services', 'Damaged Cart',
       'Low Hanging Limbs on Streets/Sidewalk', 'Scattered Material',
       'Pot Hole Repair', 'Complaint', 'Cart Exchange Request',
       'Additional Garbage Container - with Fee', 'Cart PickUp',
       '1st Cal

In [29]:
# number of unique values
df.service_request_type.nunique()

341

In [30]:
# unique values of column
df.SLA_days.unique() 

array([2.0, 999.0, 1.977453704, ..., 16.68256944, 65.67908565,
       3.662974537], dtype=object)

In [31]:
# number of unique values
df.SLA_days.nunique()

149039

In [32]:
# unique values of column
df.case_status.unique() 

array(['Closed', 'Open', 'Code Enforcement Services', ' 78210"',
       ' 78201"', ' 78223"', ' 78212"'], dtype=object)

In [33]:
# number of unique values
df.case_status.nunique()

7

In [34]:
# unique values of column
df.request_address.unique() 

array(['7059  TIMBERCREEK DR, San Antonio, 78227',
       'FREDSBG RD and NB LOOP 410 NW', 'CESAR CHAVEZ W and S ZARZAMORA',
       ..., '10270  WEYBRIDGE, San Antonio, 78250',
       'SANDY TRAIL and SPOTTED TRAIL', 'PALMETTO WAY and LEXI PETAL'],
      dtype=object)

In [35]:
# number of unique values
df.request_address.nunique()

260062

In [36]:
# unique values of column
df.council_district.unique() 

array([6, 7, 5, 1, 2, 4, 3, 10, 8, 9, 0, '2', '5', '10', '0', '1', '7',
       '8', '4', '9', '6', '3', 'YES', 'Code Enforcement Services'],
      dtype=object)

In [37]:
# number of unique values
df.council_district.nunique()

24

In [38]:
# unique values of column
df.dept_name.unique() 

array(['Solid Waste Management', 'Animal Care Services',
       'Trans & Cap Improvements', 'Code Enforcement Services',
       'Customer Service', 'Development Services', 'Parks and Recreation',
       'Metro Health', 'City Council', nan, '58', 'DSD/Code Enforcement'],
      dtype=object)

In [39]:
# number of unique values
df.dept_name.nunique()

11

In [40]:
# unique values of column
df.standardized_dept_name.unique() 

array(['Solid Waste', 'Animal Care Services', 'Trans & Cap Improvements',
       'DSD/Code Enforcement', 'Customer Service', 'Parks & Recreation',
       'Metro Health', 'City Council', 'CRM_Listener', 'YES'],
      dtype=object)

In [41]:
# number of unique values
df.standardized_dept_name.nunique()

10

In [42]:
# unique values of column
df.dept_subject_to_SLA.unique() 

array(['YES', 'NO', nan, '58'], dtype=object)

In [43]:
# number of unique values
df.dept_subject_to_SLA.nunique()

3

In [44]:
# unique values of column
df.index.unique() 

RangeIndex(start=0, stop=855269, step=1)

In [45]:
# number of unique values
df.index.nunique()

855269

In [46]:
# unique values of column
df.source_username.unique() 

array(['Isis Mendoza', 'Lori Lara', 'Amy Cardenas', 'Jessica Mendoza',
       'Michelle Urrutia', 'Mariana Rios', 'Melissa Carmona',
       'Leonard Silva', 'Socorro Quiara', 'Erica Flores',
       'San Juanita Villarreal', 'Andrew Trexler',
       'Michelle Lingenfelder', 'Jose Acosta', 'James Garcia',
       'Kimberly Green-Woods', 'Kimberly Green', 'Eva T. Kleiber',
       'Guadalupe Rodriguez', 'Andrea Alvarez', 'Tawona Martin',
       'Marisa Ozuna', 'Merlene Blodgett', 'Jerry Robles',
       'Janelle Lopez', 'Matthew Gilbert', 'Marisol Villarreal',
       'Priscilla Tillman', 'Pamela Boykin', 'Richard Sanchez',
       'Samantha McLaughlin', 'Michelle San Miguel', 'Ebony Ruffin',
       'Betty De Hoyos', 'Pamela K. Baccus', 'Leticia Garcia',
       'Renee Key', 'Beatriz Urbina', 'Refugio Campos', 'Elisa Tejeda',
       'Latreece Towers', 'Anastasia Harrison', 'Carmen Cura',
       'Jessica Pina', 'Genese Woisin', 'svcCRMSS', 'Norma K. Sotoodeh',
       'svcCRMLS', 'CRM_Listener', 

In [47]:
# number of unique values
df.source_username.nunique()

140

In [48]:
# Notes on the above. 
#### I see 'errors' in some of the values, since I do not have a data dictionary, and I not sure what some mean

In [51]:
df.case_opened_date.min(), df.case_opened_date.max()


('1/1/17 10:03', '9/9/17 9:59')

In [49]:
df.head(50)

,source_id,dept_division,case_id,case_opened_date,case_closed_date,SLA_due_date,case_late,num_days_late,case_closed,service_request_type,SLA_days,case_status,request_address,council_district,dept_name,standardized_dept_name,dept_subject_to_SLA,index,source_username
0,139344,Miscellaneous,1014127378,1/1/18 8:24,1/2/18 7:37,1/3/18 8:24,NO,-1.032870,YES,Dead Animal - Cat,2.0,Closed,"7059 TIMBERCREEK DR, San Antonio, 78227",6,Solid Waste Management,Solid Waste,YES,19,Isis Mendoza
1,139344,Field Operations,1014127380,1/1/18 8:39,1/1/18 12:30,9/26/20 8:39,NO,-998.839687,YES,Stray Animal,999.0,Closed,FREDSBG RD and NB LOOP 410 NW,7,Animal Care Services,Animal Care Services,YES,19,Isis Mendoza
2,139344,Signals,1014127389,1/1/18 9:02,1/1/18 10:12,1/3/18 8:30,NO,-1.928912,YES,Traffic Signal Ops and Maintenance - Emergency,1.977454,Closed,CESAR CHAVEZ W and S ZARZAMORA,5,Trans & Cap Improvements,Trans & Cap Improvements,YES,19,Isis Mendoza
3,139344,Streets,1014127390,1/1/18 9:04,1/3/18 8:54,1/9/18 8:30,NO,-5.983299,YES,Barricade Pick Up,7.976192,Closed,CHUPADERAS N and MORALES,1,Trans & Cap Improvements,Trans & Cap Improvements,YES,19,Isis Mendoza
4,139344,Signals,1014127394,1/1/18 9:15,1/1/18 10:54,1/3/18 8:30,NO,-1.899375,YES,Traffic Signal Ops and Maintenance - Emergency,1.968414,Closed,CALAVERAS N and FREDSBG RD,1,Trans & Cap Improvements,Trans & Cap Improvements,YES,19,Isis Mendoza
5,139344,Field Operations,1014127400,1/1/18 9:30,1/2/18 16:00,1/6/18 9:30,NO,-3.729525,YES,Animal Neglect,5.0,Closed,"254 GLOBE AVE, San Antonio, 78228",7,Animal Care Services,Animal Care Services,YES,19,Isis Mendoza
6,139344,Miscellaneous,1014127411,1/1/18 10:14,1/2/18 6:22,1/3/18 10:14,NO,-1.160556,YES,Dead Animal - Dog,2.0,Closed,"6410 IH 35 N, San Antonio, 78218",2,Solid Waste Management,Solid Waste,YES,19,Isis Mendoza
7,139344,Field Operations,1014127413,1/1/18 10:18,1/1/18 12:46,1/6/18 10:18,NO,-4.897199,YES,Aggressive Animal(Non-Critical),5.0,Closed,"700 PORTER ST, San Antonio, 78210",2,Animal Care Services,Animal Care Services,YES,19,Isis Mendoza
8,139344,Field Operations,1014127418,1/1/18 10:33,1/1/18 11:49,1/1/18 22:33,NO,-0.447269,YES,Trapped/Confined Animal,0.5,Closed,"150 RUSTLEAF DR, San Antonio, 78242",4,Animal Care Services,Animal Care Services,YES,19,Isis Mendoza
9,139344,Field Operations,1014127420,1/1/18 10:39,1/1/18 12:29,9/26/20 10:39,NO,-998.922998,YES,Stray Animal,999.0,Closed,"702 SANDMEYER, San Antonio, 78208",2,Animal Care Services,Animal Care Services,YES,19,Isis Mendoza


In [50]:
# PREP
#### get rid of nulls, arrange by date, change dates to a date format, drop some columns, rename some columns, encode columns

In [ ]:
def prep21(df):
    global df21
    df21['charter_encoded'] = df21.charter_status.map({'OPEN ENROLLMENT CHARTER': 1, 'TRADITIONAL ISD/CSD':0})
    df21=df21[(df21.heading == 'A01') | (df21.heading ==  'A03')]
    df21=df21[df21['student_count'] != '-999']
    df21['student_count']= df21['student_count'].str.replace("<", "")
    df21['student_count'] = df21['student_count'].astype(float)
    dfpivot=df21.pivot(index='campus_number', columns='heading', values= 'student_count').dropna()
    df21=df21.merge(dfpivot,how= 'right', on= 'campus_number')
    df21=df21.rename(columns={'A01': 'student_enrollment', 'A03':'discipline_count'})
    df21['discipline_percent']= ((df21['discipline_count']/df21['student_enrollment'])*100)
    df21=df21.round({'discipline_percent': 0})
    df21=df21.drop(columns=['agg_level', 'campus_number', 'region', 'charter_status', 'dist_name_num', 'student_count','section', 'heading', 'heading_name', 'student_count'])
    df21=df21.drop_duplicates()
    df21=df21.reset_index(drop=True)


# EDA